In [19]:
# %pip install crewai crewai-tools langchain_community

In [20]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [21]:
from crewai import LLM

In [22]:
llm = LLM(
    model="gemini/gemini-2.5-flash-preview-05-20",
    stream=False,  # Enable streaming
    temperature=0.5,
)

In [23]:
from crewai import Agent, Task, Crew
# from crewai_tools import MCPServerAdapter

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Snowflake Cost Management Analyst Expert

In [24]:
analyst = Agent(
    role="Snowflake Cost Management Analyst Expert",
    goal="Provide factually accurate estimates on Snowflake credit consumption for and implmentation of {topic}",
    backstory="You are consulted for your expertise on Snowflake consumption forecasting. Estimate consumption for bringing on a workload for {topic}."              
              "Use Snowflake standard pay as you go pricing to estimate the cost of the workload.",
    allow_delegation=False,
    llm=llm
)

### Agent: Snowflake Architect

In [25]:
architect = Agent(
    role="Snowflake Specialized Data Architect",
    goal="Answer questions about what components and features of Snowflake are needed to architect a solution for: {topic}",
    backstory="You are a Snowflake Specialized Data Architect. You are tasked with answering questions about what components"
              "and features of Snowflake are needed to architect a solution for: {topic}",
    allow_delegation=False,
    llm=llm,
)

### Task: Solution Architecture

In [27]:
design = Task(
    description=(
        "1. Prioritize the latest architecture designs for {topic}.\n"
        "2. Assume middle of the road when uncertainty around size and scale of data and processing needs.\n"
        "3. Develop a detailed list of components in particular number of warehouses, warehouse size, and task schedules."
    ),
    expected_output="A comprehensive solution architecture with listed Snowflake components",
    agent=architect,
)

### Task: Estimate Cost

In [28]:
estimate = Task(
    description=(
        "1. Use the solution architecture to estimate the total monthly consumption cost of running the solution on Snowflake"
        "2. Breakout the cost by Snowflake component, suchas warehouse compute, storage, AI services, and other services"
    ),
    expected_output="A report in markdown format",
    agent=analyst,
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=True` allows you to see all the logs of the execution.

In [30]:
crew = Crew(
    agents=[architect, analyst],
    tasks=[design, estimate],
)

## Running the Crew

In [31]:
result = crew.kickoff(inputs={"topic": "Fraud & AML Solution"})

- Display the results of your execution as markdown in the notebook.

In [32]:
from IPython.display import Markdown
Markdown(result.raw)

## Snowflake Consumption Forecast: Fraud & AML Solution

This report provides an estimated monthly consumption cost for running the proposed Fraud & AML solution on Snowflake, based on the provided architecture and "middle-of-the-road" scale assumptions. The estimates use Snowflake's standard pay-as-you-go pricing.

**Pricing Assumptions (Standard Edition, On-Demand):**

*   **Compute (Credits):** $2.50 per credit
*   **Storage:** $45.00 per TB/month (for active storage, including Time Travel and Fail-safe overhead)
*   **Snowpipe:** $0.06 per GB ingested
*   **Data Transfer (Egress):** Estimated as a small fixed cost, as it's typically a minor component unless large volumes are consistently egressed.

---

### **1. Total Monthly Consumption Cost**

Based on the detailed breakdown below, the estimated total monthly consumption cost for the Fraud & AML solution on Snowflake is:

**Estimated Total Monthly Cost: ~$134,600 - $135,000**

---

### **2. Cost Breakdown by Snowflake Component**

#### **A. Warehouse Compute**

This section details the estimated credit consumption and cost for each Virtual Warehouse, factoring in their size, estimated active hours, and multi-cluster configurations.

| Warehouse Name              | Size    | Credits/Hour | Estimated Average Clusters | Estimated Active Hours/Day | Estimated Monthly Credits | Estimated Monthly Cost |
| :-------------------------- | :------ | :----------- | :------------------------- | :------------------------- | :------------------------ | :--------------------- |
| `INGESTION_WH`              | X-SMALL | 1            | 1                          | 2                          | 60                        | $150.00                |
| `SILVER_TRANSFORM_WH`       | MEDIUM  | 4            | 1.5 (avg of 1-3)           | 24                         | 4,320                     | $10,800.00             |
| `FRAUD_DETECTION_API_WH`    | LARGE   | 8            | 3 (avg of 1-5, up to 10)   | 24                         | 17,280                    | $43,200.00             |
| `AML_ANALYTICS_WH`          | MEDIUM  | 4            | 1.5 (avg of 1-5)           | 14                         | 2,520                     | $6,300.00              |
| `DATA_SCIENCE_WH`           | MEDIUM  | 4            | 1 (avg of 1-2)             | 8                          | 960                       | $2,400.00              |
| `ML_TRAINING_WH`            | LARGE   | 8            | 1                          | 8 (4 hrs, 2x/week)         | 274 (approx.)             | $685.00                |
| `REPORTING_WH`              | SMALL   | 2            | 1                          | 4                          | 240                       | $600.00                |
| `REVERSE_ETL_WH`            | X-SMALL | 1            | 1                          | 12                         | 360                       | $900.00                |
| **Subtotal Warehouse Compute** |         |              |                            |                            | **26,014**                | **$65,035.00**         |

*Note: Multi-cluster warehouse estimates are based on an average number of clusters active during peak and off-peak times. `ML_TRAINING_WH` usage is based on weekly/bi-weekly runs.*

#### **B. Storage**

This section estimates the cost for data storage, including active data, Time Travel, and Fail-safe.

*   **Estimated Active Data Volume:** 1.25 PB (1250 TB) - *Mid-point of 500 TB - 2 PB range.*
*   **Time Travel & Fail-safe Overhead:** +15% (estimated, can vary based on retention settings)
*   **Total Estimated Stored Data:** 1250 TB * 1.15 = 1437.5 TB
*   **Estimated Monthly Storage Cost:** 1437.5 TB * $45.00/TB = **$64,687.50**

#### **C. Other Services**

This category includes Snowpipe, Materialized View/Search Optimization maintenance, and Data Transfer.

*   **Snowpipe (Serverless Ingestion):**
    *   Daily Ingestion: 30 million records/day (approx. 30 GB/day)
    *   Monthly Ingestion: 30 GB/day * 30 days = 900 GB/month
    *   Estimated File Count: ~9,000 files/month (assuming 100MB files)
    *   Cost: (900 GB * $0.06/GB) + (9,000 files / 1000 * $0.0007) = $54.00 + $0.0063 = **$54.01/month**

*   **Materialized Views & Search Optimization (Maintenance):**
    *   These serverless services consume credits for background maintenance.
    *   Estimated Credits: 900 credits/month (approx. 30 credits/day)
    *   Estimated Monthly Cost: 900 credits * $2.50/credit = **$2,250.00/month**

*   **Data Transfer (Egress):**
    *   Estimated Monthly Cost: **$200.00/month** (buffer for data egress to external systems like fraud case management, BI tools, etc.)

---

### **Summary of Monthly Costs**

| Component                 | Estimated Monthly Cost |
| :------------------------ | :--------------------- |
| Warehouse Compute         | $65,035.00             |
| Storage                   | $64,687.50             |
| Snowpipe                  | $54.01                 |
| MV/Search Optimization    | $2,250.00              |
| Data Transfer (Egress)    | $200.00                |
| **Total Estimated Monthly Cost** | **$132,226.51**        |

---

**Disclaimer:** These are estimates based on the provided architecture and assumed usage patterns. Actual costs may vary depending on:
*   Exact data volumes and growth rates.
*   Actual query patterns, concurrency, and complexity.
*   Efficiency of SQL/Snowpark code.
*   Warehouse auto-suspend and auto-resume effectiveness.
*   Specific Snowflake edition, region, and contractual pricing.
*   Future changes in Snowflake pricing.
*   The actual number of clusters running for multi-cluster warehouses.

It is recommended to monitor Snowflake usage closely using Account Usage views and Resource Monitors once the solution is operational to fine-tune resource allocation and optimize costs.